In [7]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import math

class LogisticRegression(object):

    def __init__(self, trainfileName, testfileName):
        self.alpha = 0.01
        self.features = 0
        
        self.read_data(trainfileName, testfileName)
        self.Beta = np.matrix( np.zeros((self.features+1))).getT()

    def read_data(self, testfile, trainfile):
        testfile = open(testfile, 'r')
        trainfile = open(trainfile, 'r')
        lines_train  = trainfile.readlines()
        lines_test = testfile.readlines()
        testfile.close()
        trainfile.close()
        self.trainsize  = len(lines_train)
        self.testsize = len(lines_test)
        len_s = 0
        i = 0
        for line in lines_train:
            s = " ".join(line.split(","))
            s = s.split()
            s = [int(x) for x in s]
            if i == 0:
                self.features = len(s)-1
                self.X_train = np.ones((self.trainsize, self.features+1),dtype=np.float32)
                self.Y_train = np.ones((self.trainsize),dtype=np.float32)
                len_s = len(s)
            for j in range(1,len_s):
                self.X_train[i, j] = s[j]
            self.Y_train[i] = s[0]
            i = i+1

        self.X_train = np.matrix(self.X_train, dtype = int)
        self.Y_train = np.matrix(self.Y_train).getT()
        
        i = 0
        for line in lines_test:
            s = " ".join(line.split(","))
            s = s.split()
            s = [int(x) for x in s]
            if i == 0:
                self.X_test = np.ones((self.testsize, self.features+1))
                self.Y_test = np.ones((self.testsize))
                len_s = len(s)
            for j in range(1,len_s):
                self.X_test[i,j] = s[j]
            self.Y_test[i] = s[0]
            i = i+1
        self.X_test = np.matrix(self.X_test)
        self.Y_test = np.matrix(self.Y_test).getT()

    def get_cost(self, weight, target, size):
        y = np.dot(weight,self.Beta)
        
        y = 1/(1+np.exp(-y))[0,:]
        label=np.zeros((len(y)))
        for i in range(0,len(y)):
            if y[i][0][0]>0.5:
                #print y[i][0]
                label[i]=1
            else:
                label[i]=0
        #print label
        #print np.sum(label==target)
        return -(np.asarray(target)*np.asarray(np.log(y))+np.asarray(1.0-target)*np.asarray(np.log(1.0-y))).mean()
        
    def get_Grad(self, X, Y):
        a = X*self.Beta
        for i in range(a.size):
            a[i,0] = 1/(1+math.exp(-a[i,0]))
        a = ((a-Y).getT())*X
        return (a*self.lr).getT()
        
    def train(self, X, Y, size):
        costList = []
        cost = self.get_cost(X,Y,size)
        print "initial cost ", cost
        prevcost = cost
        self.lr = np.identity(self.features+1)
        for i in range(self.features+1):
            self.lr[i,i] = self.alpha/(self.trainsize)
        i = 0
        while(i < 10000):
            grad = self.get_Grad(X,Y)
#            print grad
            self.Beta = self.Beta - grad
 #           print self.Beta
            cost = self.get_cost(X,Y,size)
            costList.append(cost)
            if i%100 == 0:
                print cost
            
            i = i+1
            
            if(cost >= prevcost or (cost < prevcost and abs(cost - prevcost) < 0.0000001)):
                print cost, prevcost
                break
            else:
                prevcost = cost
        return self.Beta, costList, i
   


In [4]:
def plot_learning_curve(costList, n_epoch):
    print "length of costList ", len(costList), " n_epoch ", n_epoch
    plt.plot(np.arange(n_epoch), np.array(costList))
    plt.xlabel('epochs')
    plt.ylabel('cost')
    plt.show()

In [6]:
trainf = "SPECT.train"#sys.argv[1]
testf = "SPECT.test" #sys.argv[2]
#alpha = float(sys.argv[3])

model = LogisticRegression(trainf, testf)

Beta, costList, n_epoch = model.train(model.X_train, model.Y_train, model.trainsize)



plot_learning_curve(costList, n_epoch)

initial cost  0.69314718056
0.684468821968
0.347737552975
0.29419678202
0.281568223497
0.279292499161
0.279291505074 0.279291549634
length of costList  404  n_epoch  404


/usr/lib/python2.7/site-packages/matplotlib/backends/backend_gtk.py:253: Warning: Source ID 18 was not found when attempting to remove it
  gobject.source_remove(self._idle_event_id)
